In [ ]:
import sqlite3
import json

In [ ]:
import numpy as np
import pickle

In [ ]:
con = sqlite3.connect("/mnt/infonas/data/harshiitb/MTP/MTP/FEVER/FEVEROUS-main/data/feverous_wikiv1.db")

In [ ]:
cur = con.cursor()
cur.execute("SELECT id FROM wiki;")

In [ ]:
cur.execute("SELECT id FROM wiki;")

In [ ]:
titles = np.load("/mnt/infonas/data/harshiitb/MTP/MTP/data/page_ids.pkl", allow_pickle=True)

In [ ]:
type(titles.tolist())

In [ ]:
titles[:11]

In [ ]:
titles = np.asarray(titles)

In [ ]:
titles

In [ ]:
titles[2710702]

In [ ]:
data[-1]

In [ ]:
#Now we check BLINK here

In [ ]:
import torch
torch.cuda.set_device(1)

In [1]:
import blink.main_dense as main_dense

In [2]:
import argparse

models_path = "/mnt/infonas/data/harshiitb/MTP/MTP/BLINK/models/" # the path where you stored the BLINK models

config = {
    "test_entities": None,
    "test_mentions": None,
    "interactive": False,
    "top_k": 10,
    "biencoder_model": models_path+"biencoder_wiki_large.bin",
    "biencoder_config": models_path+"biencoder_wiki_large.json",
    "entity_catalogue": models_path+"entity.jsonl",
    "entity_encoding": models_path+"all_entities_large.t7",
    "crossencoder_model": models_path+"crossencoder_wiki_large.bin",
    "crossencoder_config": models_path+"crossencoder_wiki_large.json",
    "fast": False, # set this to be true if speed is a concern
    "output_path": "logs/" # logging directory
}

args = argparse.Namespace(**config)

models = main_dense.load_models(args, logger=None)

torch.cuda.is_available():  True no cuda:  False
Device found:  cuda has  1 GPUs
torch.float32
Inside cross_encoder and it is using:  cuda


In [3]:
text = ['"900", Cahiers d\'Italie et d\'Europe History The first four preambles, Giustification, Basis, Advices, Analogies were published in "French" in the journals of autumn 1926, March and June 1927.']

In [ ]:
text = ["Hi, sachin visited India"]

In [4]:
import blink.ner as NER

In [5]:
ner_model = NER.get_model()

2022-01-15 15:54:02,840 --------------------------------------------------------------------------------
2022-01-15 15:54:02,843 The model key 'ner-fast' now maps to 'https://huggingface.co/flair/ner-english-fast' on the HuggingFace ModelHub
2022-01-15 15:54:02,844  - The most current version of the model is automatically downloaded from there.
2022-01-15 15:54:02,845  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/ner-fast/en-ner-fast-conll03-v0.4.pt)
2022-01-15 15:54:02,846 --------------------------------------------------------------------------------
2022-01-15 15:54:03,760 loading file /mnt/b100/d0/harshiitb/.flair/models/ner-english-fast/4c58e7191ff952c030b82db25b3694b58800b0e722ff15427f527e1631ed6142.e13c7c4664ffe2bbfa8f1f5375bd0dced866b8c1dd7ff89a6d705518abf0a611


In [6]:
annotated = main_dense._annotate(ner_model,text)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 2945.44it/s]


In [7]:
annotated

[{'label': 'unknown',
  'label_id': -1,
  'context_left': '"900", ',
  'context_right': ' the first four preambles, giustification, basis, advices, analogies were published in "french" in the journals of autumn 1926, march and june 1927.',
  'mention': "cahiers d'italie et d'europe history",
  'start_pos': 7,
  'end_pos': 43,
  'sent_idx': 0},
 {'label': 'unknown',
  'label_id': -1,
  'context_left': '"900", cahiers d\'italie et d\'europe history the first four preambles, ',
  'context_right': ', basis, advices, analogies were published in "french" in the journals of autumn 1926, march and june 1927.',
  'mention': 'giustification',
  'start_pos': 70,
  'end_pos': 84,
  'sent_idx': 0},
 {'label': 'unknown',
  'label_id': -1,
  'context_left': '"900", cahiers d\'italie et d\'europe history the first four preambles, giustification, ',
  'context_right': ', advices, analogies were published in "french" in the journals of autumn 1926, march and june 1927.',
  'mention': 'basis',
  'start_p

In [ ]:
window_ids = [0,1,2]
window_density = dict.fromkeys(window_ids, 0)
for ent in annotated:
    window_density[ent["sent_idx"]] += 1
window_density

In [8]:
_, _, _, _, _, predictions2, scores,_ = main_dense.run(args, None, *models, test_data=annotated)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.44it/s]


Debug Step 1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 1258.34it/s]

Debug Step 2


Cross Encoder about to start!


Evaluation: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.79it/s]


In [9]:
predictions2

[['History of Europe',
  'History of France',
  'Atlantic history',
  'Grandes Chroniques de France',
  'Timeline of the Middle Ages',
  'Augustan History',
  'History of Corsica',
  'The Oxford History of Early Modern Europe',
  'History of the Lombards',
  'Fontana History of European War and Society'],
 ['Codification (law)',
  'Codification (linguistics)',
  'Giselle',
  'Liberature',
  'Glorification',
  'Clementine literature',
  'Gisela (name)',
  'List of Greek and Latin roots in English/J',
  'Giselher',
  'Studentification'],
 ['Primary source',
  'Historical source',
  'Source text',
  'Basis of estimate',
  'Foundation (evidence)',
  'Groundwork of the Metaphysic of Morals',
  'Foundations of mathematics',
  'Foundations of the Science of Knowledge',
  'Foundations of Leninism',
  'Basis theorem'],
 ['Analogy',
  'Metaphor',
  'Analogue (literature)',
  'Imitation',
  'Argument from analogy',
  'Reference',
  'Figurative analogy',
  'Approximation',
  'Cf.',
  'Analogical c

In [ ]:
scores

In [ ]:
inverted_index = {}
index = 0
for w_id in range(len(text)):
    for _ in range(window_density[w_id]):
        try:
            inverted_index[predictions[index][0]].append(w_id)
        except KeyError:
            inverted_index[predictions[index][0]] = []
            inverted_index[predictions[index][0]].append(w_id)
        index += 1
inverted_index
            

In [ ]:
#Now try ELQ

In [ ]:
import elq.main_dense as elq_main_dense
import sys
import argparse

In [ ]:
sys.path.append("/mnt/infonas/data/harshiitb/MTP/MTP/BLINK")

In [ ]:
sys.path

In [ ]:
models_path = "/mnt/infonas/data/harshiitb/MTP/MTP/BLINK/models/"

In [ ]:
elq_config = {
    "interactive": False,
    "biencoder_model": models_path+"elq_wiki_large.bin",
    "biencoder_config": models_path+"elq_large_params.txt",
    "cand_token_ids_path": models_path+"entity_token_ids_128.t7",
    "entity_catalogue": models_path+"entity.jsonl",
    "entity_encoding": models_path+"all_entities_large.t7",
    "output_path": "logs/", # logging directory
    "faiss_index": "hnsw",
    "index_path": models_path+"faiss_hnsw_index.pkl",
    "num_cand_mentions": 10,
    "num_cand_entities": 10,
    "threshold_type": "joint",
    "threshold": -4.5,
}

elq_args = argparse.Namespace(**elq_config)

elq_models = elq_main_dense.load_models(elq_args, logger=None)

In [ ]:
data_to_link = [{
                    "id": 0,
                    "text": "paris is capital of which country?".lower(),
                },
                {
                    "id": 1,
                    "text": "paris is great granddaughter of whom?".lower(),
                },
                {
                    "id": 2,
                    "text": "Sachin Tendulkar visited India".lower(),
                },
                ]

predictions = elq_main_dense.run(elq_args, None, *elq_models, test_data=data_to_link)

In [ ]:
predictions

In [ ]:
#Now Let's check high IDF

In [ ]:
import pandas as pd

In [ ]:
dataframe = pd.read_csv("/mnt/infonas/data/harshiitb/MTP/MTP/data/wiki_tfidf_stems.csv")

In [ ]:
import snowballstemmer
stemmer = snowballstemmer.stemmer('english')

In [ ]:
dataframe[:5]

In [ ]:
s = stemmer.stemWord("nominated")

In [ ]:
print(s)

In [ ]:
df = dataframe[dataframe['idf']<=7]
df = df[df['idf']>=3]

In [ ]:
df = df[["stem","most_freq_term"]]

In [ ]:
df

In [ ]:
rare_words = df.set_index('stem').T.to_dict('list')

In [ ]:
rare_words

In [ ]:
#Now load the arae_word.pkl

In [ ]:
with open("/mnt/infonas/data/harshiitb/MTP/MTP/data/rare_words.pkl", "rb") as file:
    rw = pickle.load(file)

In [ ]:
len(rw.keys())

In [ ]:
pwd

In [ ]:
index = np.load("/mnt/infonas/data/harshiitb/MTP/MTP/data/idx.pkl", allow_pickle=True)

In [ ]:
index['Australia']

In [ ]:
index.keys()

In [ ]:
index1 = np.load("/mnt/infonas/data/harshiitb/MTP/MTP/data/idx.pkl", allow_pickle=True)

In [ ]:
index1.keys()

In [ ]:
index1['Australia']

In [ ]:
index2 = np.load("/mnt/infonas/data/harshiitb/MTP/MTP/data/idx1.pkl", allow_pickle=True)

In [ ]:
index2.keys()

In [ ]:
index2['Portugal']

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

In [ ]:
# load tagger
tagger = SequenceTagger.load("flair/ner-english")

In [ ]:
text = ["George Washington went to Washington","Virat Kohli scored a century."]
sens = []
for sen in text:
    sens.append(Sentence(sen,use_tokenizer=True))

In [ ]:
sens

In [ ]:
tagger.predict(sens)

In [ ]:
sent_mentions = sens[1].to_dict(tag_type="ner")

In [ ]:
sent_mentions